In [1]:
import pandas as pd
import os
import shutil
from thyroid.utils.exception import customException

In [2]:
print("Current working directory:", os.getcwd())

Current working directory: c:\Users\aakas\Documents\Projects\Thyroid-disease-detection\research


In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\aakas\\Documents\\Projects\\Thyroid-disease-detection'

In [22]:
# define configuration
raw_directory = "raw_data"
artifact_directory = "artifact"
file_name = "allbp.data"
bad_directory = "bad_data"
ingested_directory = "ingested_data"

In [6]:
# Create the artifact directory if it doesn't exist
if not os.path.exists(artifact_directory):
    os.makedirs(artifact_directory)

# Create the ingested_data directory inside the artifact directory if it doesn't exist
ingested_data_directory = os.path.join(artifact_directory, ingested_directory)
if not os.path.exists(ingested_data_directory):
    os.makedirs(ingested_data_directory)


# Create the ingested_data directory inside the artifact directory if it doesn't exist
bad_data_directory = os.path.join(artifact_directory, bad_directory)
if not os.path.exists(bad_data_directory):
    os.makedirs(bad_data_directory)


In [7]:
import os

def read_data_file(file_path):
    if file_path.lower().endswith('.data' or '.txt'):
        try:
            with open(file_path, 'r') as file:
                data = file.readlines()
                print("read_data_file",)
                return data
            
        except FileNotFoundError:
            return "File not found."
        except Exception as e:
            return "An error occurred: " + str(e)
    else:
        # if the file does not end with .data
        return False


In [8]:
import csv
from datetime import datetime

In [9]:
def save_to_csv(data, csv_path):
    try:
        print(len(data))
        with open(csv_path, 'w', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            for row in data:
                csv_writer.writerow(row)
        return "CSV file saved successfully."
    except Exception as e:
        return "An error occurred while saving CSV: " + str(e)

In [ ]:
with open(csv_file_name, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    for row in processed_data:
        csv_writer.writerow(row)

In [10]:
import re
def process_rows(record):
    cleaned_attributes = []
    for attribute in record.strip().split(','):
        attribute = re.sub(r'-', 'negative', attribute)  # Replace '-' with 'negative'
        if '.|' in attribute:
            cleaned_attributes.append(attribute.split('.|')[0])
        elif '[' in attribute:
            cleaned_attributes.append(attribute.split('[')[0])
        else:
            cleaned_attributes.append(attribute)
    return cleaned_attributes

In [23]:
# process the file extension
file_path = os.path.join(raw_directory, file_name)
data_rows = read_data_file(file_path)

read_data_file


In [16]:
from datetime import datetime

In [24]:
processed_data = []
if data_rows:
    print("Processing:", file_name)
    for row in data_rows:
        processed_data.append(process_rows(row))

        
    # convert file into .csv and save it to artifacts/ingested directory
    current_datetime = datetime.now().strftime('%Y%m%d_%H%M%S')
    csv_file_name = f"thyroid_{current_datetime}.csv"
    csv_path = os.path.join(ingested_data_directory, csv_file_name)
    save_result = save_to_csv(processed_data, csv_path)
else:
    bad_file_path = os.path.join(bad_data_directory, file_name)
    shutil.move(file_path, bad_file_path)
    print("Moved to bad_data:", file_name)     

Processing: allbp.data
2800
